# Phase 1 Project

![TakeOff](./images/take_off.jpg)

## 1.  Importing and loading data sets, initial inspection

In [205]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('Data\AviationData.csv', encoding='latin-1', low_memory=False)
states_df = pd.read_csv('Data\\USState_Codes.csv')
#seats_df = pd.read_csv('Data\\AircraftSeats.csv')

In [206]:
# code the state names and abbreviations into the main df
states_dict = {}
for index, row in states_df.iterrows():
    states_dict[row[1]] = row[0]
    
    
def state_lookup(abbrev):
    if abbrev in states_dict.keys():
        return states_dict[abbrev]
    else:
        return None

df['State.Name'] = df['Location']
df['State.Abbrev'] = df['Location'].map(lambda x: str(x)[-2:])
df['State.Name'] = df['State.Abbrev'].map(lambda x: state_lookup(x) if state_lookup(x) != None else x)



In [207]:
# code the number of seats for 'Model'
#seats_dict = pd.Series(seats_df['Seating Capacity'].values, index=seats_df['Model']).to_dict()
#seats_dict
#df['Seating.Capacity'] = df['Model'].map(lambda x: seats_dict[x] if x in seats_dict.keys() else '')

In [208]:
df['State.Name'].value_counts().head()

California    8882
Texas         5913
Florida       5825
Alaska        5672
Arizona       2834
Name: State.Name, dtype: int64

In [209]:
# take a look at the value counts
for column in df.columns:
    print(f'Column Name: {column}')
    print(f'{df[column].value_counts()} \n')

Column Name: Event.Id
20001212X19172    3
20001214X45071    3
20001212X22245    2
20001214X40502    2
20001212X21286    2
                 ..
20001213X25745    1
20040310X00312    1
20001214X42385    1
20001211X14064    1
20070403X00370    1
Name: Event.Id, Length: 87951, dtype: int64 

Column Name: Investigation.Type
Accident    85015
Incident     3874
Name: Investigation.Type, dtype: int64 

Column Name: Accident.Number
DCA22WA172    2
GAA22WA241    2
WPR23LA045    2
ERA22FA338    2
WPR23LA041    2
             ..
NYC84LA181    1
BFO95LA083    1
ENG11WA038    1
CEN15LA018    1
WPR18FA141    1
Name: Accident.Number, Length: 88863, dtype: int64 

Column Name: Event.Date
1982-05-16    25
2000-07-08    25
1984-06-30    25
1986-05-17    24
1983-08-05    24
              ..
2015-10-28     1
2021-03-08     1
2021-02-06     1
2021-11-16     1
2013-09-17     1
Name: Event.Date, Length: 14782, dtype: int64 

Column Name: Location
ANCHORAGE, AK            434
MIAMI, FL                200
ALBUQU

0.0      61454
1.0      10320
2.0       3576
3.0        784
4.0        372
5.0        129
6.0         67
7.0         59
9.0         22
8.0         20
13.0        14
12.0        11
10.0        11
14.0        10
11.0         9
17.0         8
18.0         6
19.0         6
22.0         5
24.0         5
15.0         4
33.0         4
16.0         4
25.0         4
23.0         3
21.0         3
32.0         3
20.0         3
27.0         3
26.0         3
30.0         2
36.0         2
42.0         2
28.0         2
38.0         2
50.0         2
31.0         2
43.0         1
39.0         1
65.0         1
47.0         1
57.0         1
58.0         1
29.0         1
45.0         1
62.0         1
71.0         1
200.0        1
125.0        1
96.0         1
69.0         1
380.0        1
68.0         1
171.0        1
35.0         1
40.0         1
84.0         1
Name: Total.Minor.Injuries, dtype: int64 

Column Name: Total.Uninjured
0.0      29879
1.0      25101
2.0      15988
3.0       4313
4.0       266

## 1. Data Cleaning

In [210]:
#Dropping these columns due to Not enough data and/or not enough relevance
drop_columns = ['Latitude',          
                'Longitude',         
                'Schedule',          
                'Air.carrier',       
                'Airport.Code',      
                'FAR.Description',   
                'Publication.Date',  
                'Report.Status',
                'Accident.Number',
                'Registration.Number'
               ]
df = df.drop(columns=drop_columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Country                 88663 non-null  object 
 5   Airport.Name            52790 non-null  object 
 6   Injury.Severity         87889 non-null  object 
 7   Aircraft.damage         85695 non-null  object 
 8   Aircraft.Category       32287 non-null  object 
 9   Make                    88826 non-null  object 
 10  Model                   88797 non-null  object 
 11  Amateur.Built           88787 non-null  object 
 12  Number.of.Engines       82805 non-null  float64
 13  Engine.Type             81812 non-null  object 
 14  Purpose.of.flight       82697 non-null

In [211]:
#Take a look at amateur built values
df['Amateur.Built'].value_counts()


No     80312
Yes     8475
Name: Amateur.Built, dtype: int64

In [212]:
#Take a look at amateur built NaN
df['Amateur.Built'].isna().sum()

102

In [213]:
# Stakeholders will not want to purchase airplanes built by amateurs for liability reasons
# dropping 'Amateur.Built' == 'Yes' or NaN 
df = df[(df['Amateur.Built'] == 'No') & (df['Amateur.Built'].notna())]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80312 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                80312 non-null  object 
 1   Investigation.Type      80312 non-null  object 
 2   Event.Date              80312 non-null  object 
 3   Location                80265 non-null  object 
 4   Country                 80092 non-null  object 
 5   Airport.Name            47400 non-null  object 
 6   Injury.Severity         79313 non-null  object 
 7   Aircraft.damage         77165 non-null  object 
 8   Aircraft.Category       28721 non-null  object 
 9   Make                    80266 non-null  object 
 10  Model                   80245 non-null  object 
 11  Amateur.Built           80312 non-null  object 
 12  Number.of.Engines       74606 non-null  float64
 13  Engine.Type             73648 non-null  object 
 14  Purpose.of.flight       74242 non-null

In [214]:
# looking at 'Investigation.Type'
df['Investigation.Type'].value_counts()

Accident    76520
Incident     3792
Name: Investigation.Type, dtype: int64

In [215]:
# see what the difference is in 'Investigation.Type'
df.groupby('Investigation.Type')[['Total.Fatal.Injuries', 'Total.Uninjured']].describe()

Total.Fatal.Injuries                                     \
                                  count      mean       std  min  25%  50%   
Investigation.Type                                                           
Accident                        66952.0  0.695782  5.851423  0.0  0.0  0.0   
Incident                         3203.0  0.006556  0.094941  0.0  0.0  0.0   

                               Total.Uninjured                             \
                    75%    max           count       mean        std  min   
Investigation.Type                                                          
Accident            0.0  349.0         71746.0   3.467538  20.626871  0.0   
Incident            0.0    3.0          3679.0  50.334874  83.425972  0.0   

                                           
                    25%  50%   75%    max  
Investigation.Type                         
Accident            0.0  1.0   2.0  699.0  
Incident            1.0  4.0  78.0  588.0

In [216]:
# seems like Incident tends to be much less serious, dont want to confound the data
# dropping Incident from Investigation.Type
#df = df[df['Investigation.Type'] == 'Accident']

In [217]:
# Take a closer look at country
df['Country'].value_counts()


United States           73906
Brazil                    367
Mexico                    348
Canada                    346
United Kingdom            327
                        ...  
French Guiana               1
Scotland                    1
Corsica                     1
Ivory Coast                 1
United Arab Emirates        1
Name: Country, Length: 217, dtype: int64

In [218]:
# Vast majority of this data set is from US crashes
# we shouldn't assume the model fits outside of US, removing non-US
df = df[df['Country'] == 'United States']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73906 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                73906 non-null  object 
 1   Investigation.Type      73906 non-null  object 
 2   Event.Date              73906 non-null  object 
 3   Location                73896 non-null  object 
 4   Country                 73906 non-null  object 
 5   Airport.Name            46285 non-null  object 
 6   Injury.Severity         73798 non-null  object 
 7   Aircraft.damage         71949 non-null  object 
 8   Aircraft.Category       24697 non-null  object 
 9   Make                    73897 non-null  object 
 10  Model                   73891 non-null  object 
 11  Amateur.Built           73906 non-null  object 
 12  Number.of.Engines       72237 non-null  float64
 13  Engine.Type             71156 non-null  object 
 14  Purpose.of.flight       71481 non-null

In [219]:
# in 'Injury.Severity', when Fatal(x), x is just = Total.Fatal.Injuries
# so recoding all Fatal(x) to just Fatal
df['Injury.Severity'] = df['Injury.Severity'].map(lambda x: 'Fatal' if str(x)[:5] == 'Fatal' else x)
df['Injury.Severity'].value_counts()

Non-Fatal      58856
Fatal          12763
Incident        1851
Minor            181
Serious          133
Unavailable       14
Name: Injury.Severity, dtype: int64

In [220]:
# A lot of missing data in these catagorical columns but may be relevant data,
# so recoding the NaN as 'DATA MISSING' for now
missing_data_dict = {'Airport.Name': 'DATA MISSING',
                     'Aircraft.Category': 'DATA MISSING',
                     'Broad.phase.of.flight': 'DATA MISSING',
                     'Purpose.of.flight': 'DATA MISSING'}
df = df.fillna(missing_data_dict)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73906 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                73906 non-null  object 
 1   Investigation.Type      73906 non-null  object 
 2   Event.Date              73906 non-null  object 
 3   Location                73896 non-null  object 
 4   Country                 73906 non-null  object 
 5   Airport.Name            73906 non-null  object 
 6   Injury.Severity         73798 non-null  object 
 7   Aircraft.damage         71949 non-null  object 
 8   Aircraft.Category       73906 non-null  object 
 9   Make                    73897 non-null  object 
 10  Model                   73891 non-null  object 
 11  Amateur.Built           73906 non-null  object 
 12  Number.of.Engines       72237 non-null  float64
 13  Engine.Type             71156 non-null  object 
 14  Purpose.of.flight       73906 non-null

In [221]:
# Take a closer look at 'Aircraft.Catagory' due to relevance
df['Aircraft.Category'].value_counts()


DATA MISSING         49209
Airplane             21121
Helicopter            2593
Glider                 472
Balloon                227
Weight-Shift           139
Powered Parachute       82
Gyrocraft               31
Ultralight              13
WSFT                     9
Blimp                    4
Powered-Lift             3
Unknown                  2
Rocket                   1
Name: Aircraft.Category, dtype: int64

In [222]:
# Not much data outside of Airplane and Helicopter, and stakeholders want high volume
# for revenue, so dropping all other 'Aircraft.Category'
df = df[(df['Aircraft.Category'] == 'Airplane') | 
        (df['Aircraft.Category'] == 'Helicopter') | 
        (df['Aircraft.Category'] == 'DATA MISSING')]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72923 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                72923 non-null  object 
 1   Investigation.Type      72923 non-null  object 
 2   Event.Date              72923 non-null  object 
 3   Location                72913 non-null  object 
 4   Country                 72923 non-null  object 
 5   Airport.Name            72923 non-null  object 
 6   Injury.Severity         72815 non-null  object 
 7   Aircraft.damage         71069 non-null  object 
 8   Aircraft.Category       72923 non-null  object 
 9   Make                    72914 non-null  object 
 10  Model                   72908 non-null  object 
 11  Amateur.Built           72923 non-null  object 
 12  Number.of.Engines       71400 non-null  float64
 13  Engine.Type             70741 non-null  object 
 14  Purpose.of.flight       72923 non-null

In [223]:
# looking at the stats for the continuous data, for possible imputing
df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,71400.000000,63579.000000,62962.000000,63563.000000,68884.000000
mean,1.158473,0.425046,0.252899,0.337932,4.723477
std,0.436469,2.570353,1.195201,1.371639,25.088742
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,265.000000,137.000000,125.000000,699.000000


In [224]:
#imputing 0 for the injuries columns (due to mean, Q3 = 0)
#missing_data_dict = {'Total.Fatal.Injuries': 0,
#                     'Total.Serious.Injuries': 0,
#                     'Total.Minor.Injuries': 0,}
#df = df.fillna(missing_data_dict)

#not sure about imputing for Total.Uninjured, dropping those rows
#df = df[df['Total.Uninjured'].notna()]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72923 entries, 0 to 88888
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                72923 non-null  object 
 1   Investigation.Type      72923 non-null  object 
 2   Event.Date              72923 non-null  object 
 3   Location                72913 non-null  object 
 4   Country                 72923 non-null  object 
 5   Airport.Name            72923 non-null  object 
 6   Injury.Severity         72815 non-null  object 
 7   Aircraft.damage         71069 non-null  object 
 8   Aircraft.Category       72923 non-null  object 
 9   Make                    72914 non-null  object 
 10  Model                   72908 non-null  object 
 11  Amateur.Built           72923 non-null  object 
 12  Number.of.Engines       71400 non-null  float64
 13  Engine.Type             70741 non-null  object 
 14  Purpose.of.flight       72923 non-null

In [225]:
# get the total number of passengers per flight
df['Total.Passengers'] = (df['Total.Fatal.Injuries'] + df['Total.Serious.Injuries'] + 
                          df['Total.Minor.Injuries'] + df['Total.Uninjured'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72923 entries, 0 to 88888
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                72923 non-null  object 
 1   Investigation.Type      72923 non-null  object 
 2   Event.Date              72923 non-null  object 
 3   Location                72913 non-null  object 
 4   Country                 72923 non-null  object 
 5   Airport.Name            72923 non-null  object 
 6   Injury.Severity         72815 non-null  object 
 7   Aircraft.damage         71069 non-null  object 
 8   Aircraft.Category       72923 non-null  object 
 9   Make                    72914 non-null  object 
 10  Model                   72908 non-null  object 
 11  Amateur.Built           72923 non-null  object 
 12  Number.of.Engines       71400 non-null  float64
 13  Engine.Type             70741 non-null  object 
 14  Purpose.of.flight       72923 non-null

In [226]:
# check out the data chronologically
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
df['Event.Date'].dt.year.value_counts()


1982    3245
1983    3236
1984    3190
1985    2882
1986    2644
1987    2617
1988    2494
1989    2323
1990    2278
1991    2219
1992    2052
1993    2041
1995    1993
1994    1981
1996    1900
1999    1845
1997    1839
1998    1830
2000    1798
2003    1710
2001    1676
2002    1646
2005    1589
2007    1555
2004    1551
2008    1422
2006    1416
2011    1328
2010    1300
2009    1286
2012    1282
2016    1123
2013    1116
2018    1102
2017    1101
2019    1087
2015    1084
2022    1083
2014    1070
2021    1034
2020     948
1979       2
1977       1
1948       1
1981       1
1962       1
1974       1
Name: Event.Date, dtype: int64

In [227]:
#drop the very old one-offs, questionable data
df = df[df['Event.Date'].dt.year > 1981]

In [228]:
# maybe theres something interesting about crash month, making a column for it
df['Event.Month'] = df['Event.Date'].dt.month
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72916 entries, 7 to 88888
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                72916 non-null  object        
 1   Investigation.Type      72916 non-null  object        
 2   Event.Date              72916 non-null  datetime64[ns]
 3   Location                72906 non-null  object        
 4   Country                 72916 non-null  object        
 5   Airport.Name            72916 non-null  object        
 6   Injury.Severity         72808 non-null  object        
 7   Aircraft.damage         71062 non-null  object        
 8   Aircraft.Category       72916 non-null  object        
 9   Make                    72907 non-null  object        
 10  Model                   72901 non-null  object        
 11  Amateur.Built           72916 non-null  object        
 12  Number.of.Engines       71394 non-null  float6

In [229]:
# cleaning up 'Make' and 'Model' to standardize catagorical values
df['Make'] = df['Make'].str.upper()
df['Model'] = df['Model'].str.replace(' ', '')

In [230]:
#Save out the reduced file for tableau visualization
df.to_csv('Data\AviationDataReduced.csv')

In [231]:
#df.groupby(['Sex', 'Pclass'])['Survived'].mean()
model_df = df.groupby(['Make', 'Model'])['Total.Fatal.Injuries'].sum().sort_values(ascending=False)
#model_df = df.groupby(['Model'])['Total.Uninjured'].sum().sort_values(ascending=False)
#model_df.to_csv('Data\Models.csv')
#model_df